# **Create spatial data, web map it, reproject it and export it with Python**

This tutorial shows how to:

* create a spatial layer from a .csv file with X,Y coordinates
* display spatial layers in an interactive web map
* reproject a spatial layer to a different coordinate reference system
* export a vector layers to common geospatial formats

## **Create a spatial point layer from coordinates in a .csv file**

In this first exercise let's use the X and Y coordinates from a .csv file to generate a point layer.

Import the pandas library as pd and also the geopandas library:

In [ ]:
import pandas as pd
import geopandas

Read in a .csv file with pandas' `read_csv()` function entering the name of the file (stations_IDEAM_sept_1_2016.csv):

In [ ]:
xy_table = pd.read_csv('stations_IDEAM_sept_1_2016.csv')

The `xy_table` DataFrame has two columns with x- and y-coordinates ('LONGITUD' and 'LATITUD', respectively):

In [ ]:
xy_table.head()

To create a points layer from the `xy_table` first put the coordinate values together in one column ('coordinates'):

* Select the 'LONGITUD' and 'LATITUD' columns from `xy_table`
* Extract the `values` from the two columns as an array 
* Convert the values array to list

In [ ]:
xy_table['coordinates'] = xy_table[['LONGITUD', 'LATITUD']].values.tolist()
xy_table.head()

Change each record in the 'coordinates' column in `xy_table` to class Point:
*  Import the Point class from the geometry module in the shapely library
*  Then run `apply()` to change the coordinates to class Point

In [ ]:
from shapely.geometry import Point
xy_table['coordinates'] = xy_table['coordinates'].apply(Point)
xy_table.head()

`xy_table` is still a pandas DataFrame:

In [ ]:
type(xy_table)

To convert `xy_table` to a spatial object use the `GeoDataFrame` constructor from geopandas entering the DataFrame and the column with the geometry data as inputs:

In [ ]:
stations = geopandas.GeoDataFrame(xy_table, geometry = 'coordinates')
type(stations)

Plot the `stations` GeoDataFrame (static plot):

In [ ]:
stations.plot()

## **Display a point layer in a leaflet-based web map**

A web map can be easily created with the folium library. 

For instructions on installing folium visit: https://python-visualization.github.io/folium/installing.html#installation

After installation import the folium library:

In [ ]:
import folium

To create a leaflet web map use `folium.Map()` entering a list with latitude and longitude coordinates for the 'location' parameter (note that the order is [lat,long], not [long,lat]).

For example enter the average values for the 'LATITUD' and 'LONGITUD' columns in the `stations` GeoDataFrame (optionally enter a starting zoom level):

In [ ]:
osm_map = folium.Map(location=[stations.LATITUD.mean(), stations.LONGITUD.mean()], zoom_start=7)
osm_map

To add a GeoDataFrame to the folium map, create a GeoJSON representation of the GeodataFrame, and then use `folium.features.GeoJson()` to create a GeoJson object for plotting into the map:

In [ ]:
points = folium.features.GeoJson(stations.to_json())

Then add the GeoJson object called `points` to the `map_osm` map with `add_child()`: 

In [ ]:
osm_map.add_child(points)
osm_map

## **Reproject a spatial layer with Python**

To reproject a layer, the layer must have its coordinate reference system (CRS) defined, otherwise it's not possible to do the reprojection.

Check whether the GeoDataFrame has a defined CRS:

In [ ]:
stations.crs

In [ ]:
stations.crs is None

If a layer does not have a CRS defined yet, use the `set_crs()` method to define it appropriately: 

In [ ]:
stations = stations.set_crs('epsg:4326')
stations.crs

Reproject the `stations` layer to a different CRS using `to_crs()` for example entering an EPSG code for the desired reprojection (e.g., 'EPSG:3117'):

In [ ]:
stations_epsg3117 = stations.to_crs('EPSG:3117')
stations_epsg3117.crs

## **Export a GeoDataFrame to common GIS formats**

Use the `to_file()` method from geopandas to write a GeoDataFrame to a file in any of more than 20 possible formats including GeoPackage, GeoJSON, GML, ESRI Shapefile, and others:

In [ ]:
stations.to_file('exported_file.shp')